In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from LSTM import *
import pandas as pd
import numpy as np
import torch.utils.data as utils
import time
import os
import pickle

import pdb

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

torch.manual_seed(1)

## TRAIN SET

In [2]:
gt_train_coord = pickle.load( open( "../import_dataset_3/train/gt_train_coord.pkl", "rb" ) )
gt_train  = pickle.load( open( "../import_dataset_3/train/gt_train.pkl", "rb" ) )
in_train_coord  = pickle.load( open( "../import_dataset_3/train/in_train_coord.pkl", "rb" ) )
inputs_train = pickle.load( open( "../import_dataset_3/train/inputs_train.pkl", "rb" ) )


inputs_train = inputs_train[:,:,2:]
gt_train = gt_train[:,:,2:]

inputs_train = torch.from_numpy(inputs_train).float()
gt_train = torch.from_numpy(gt_train).float()
gt_train_coord.shape,gt_train.shape,in_train_coord.shape,inputs_train.shape

((11, 5157, 2),
 torch.Size([5157, 11, 2]),
 (10, 5157, 2),
 torch.Size([5157, 10, 242]))

## VALIDATION SET

In [3]:
gt_validation_coord = pickle.load( open( "../import_dataset_3/validation/gt_validation_coord.pkl", "rb" ) )
gt_validation = pickle.load( open( "../import_dataset_3/validation/gt_validation.pkl", "rb" ) )
in_validation_coord = pickle.load( open( "../import_dataset_3/validation/in_validation_coord.pkl", "rb" ) )
inputs_validation = pickle.load( open( "../import_dataset_3/validation/inputs_validation.pkl", "rb" ) )

inputs_validation = inputs_validation[:,:,2:]
gt_validation = gt_validation[:,:,2:]

inputs_validation = torch.from_numpy(inputs_validation).float()
gt_validation = torch.from_numpy(gt_validation).float()

gt_validation_coord.shape,gt_validation.shape,in_validation_coord.shape,inputs_validation.shape

((11, 1719, 2),
 torch.Size([1719, 11, 2]),
 (10, 1719, 2),
 torch.Size([1719, 10, 242]))

### Model

In [6]:
from torch import optim

inp_size = np.int(inputs_train.shape[2]/4)
lr = 0.01
lstm = LSTM(input_size = inputs_train.shape[2], output_size = 2, num_layers=2, hidden_size=128)
optimizer = optim.Adam(lstm.parameters(), lr=lr, weight_decay=1e-4)
criterion = nn.MSELoss()

In [ ]:
traindataset = utils.TensorDataset(inputs_train, gt_train[:,1:,:])
trainloader = utils.DataLoader(traindataset, batch_size=16, shuffle=True)

valdataset = utils.TensorDataset(inputs_validation, gt_validation[:,1:,:])
valloader = utils.DataLoader(valdataset, batch_size=16, shuffle=True)

epochs = 100
steps = 0
print_every = 323
running_loss = 0 #### DOD

loss_train = []
loss_val = []
for e in range(epochs):
    start = time.time()
    total_train_loss=0
    steps_bis = 0
    if (e+1)%10==0:
        lr /= 2
        optimizer = optim.Adam(lstm.parameters(), lr=lr,weight_decay=1e-4)

    for train_coord, ground_tru in iter(trainloader):
        steps += 1
        steps_bis+=1
        
        train_coord = train_coord.permute([1,0,2])
        ground_tru = ground_tru.permute([1,0,2])

        in_train = Variable(train_coord)
        targets = Variable(ground_tru)
        optimizer.zero_grad()
        #print(in_train.shape)
        #print(targets.shape)
        out = lstm.forward(in_train)
    
        #pdb.set_trace()
        loss1 = (criterion(out[:,:,0:2], targets[:,:,0:2]))

        loss1.backward()
        optimizer.step()
        
        running_loss += (loss1).item()
        total_train_loss += (loss1).item()
    
        if steps % print_every == 0:
                stop = time.time()
                val_loss=0
                for ii, (valcoord, valgt) in enumerate(valloader):
                    valcoord = valcoord.permute([1,0,2])
                    valgt = valgt.permute([1,0,2])
                    inputs = Variable(valcoord, volatile=True)
                    predicted = lstm.predict(inputs)
                    
                    val_loss+= (criterion(predicted[:,:,0:2],valgt[:,:,0:2]).item())
                    
                print("Epoch: {}/{}..".format(e+1, epochs),
                  "Validation loss: {:.4f}..".format(val_loss/ii),
                  "Training loss: {:.4f}..".format(running_loss/print_every),
                  "{:.4f} s/batch".format((stop - start)/print_every)
                 )
                loss_val.append(val_loss/ii)
                running_loss = 0
                start = time.time()
    loss_train.append(total_train_loss/steps_bis)

/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Epoch: 1/100.. Validation loss: 0.1226.. Training loss: 0.1356.. 0.0305 s/batch
Epoch: 2/100.. Validation loss: 0.1275.. Training loss: 0.1196.. 0.0282 s/batch
Epoch: 3/100.. Validation loss: 0.1201.. Training loss: 0.1158.. 0.0297 s/batch
Epoch: 4/100.. Validation loss: 0.1200.. Training loss: 0.1122.. 0.0363 s/batch
Epoch: 5/100.. Validation loss: 0.1216.. Training loss: 0.1106.. 0.0315 s/batch
Epoch: 6/100.. Validation loss: 0.1176.. Training loss: 0.1084.. 0.0316 s/batch
Epoch: 7/100.. Validation loss: 0.1140.. Training loss: 0.1078.. 0.0332 s/batch
Epoch: 8/100.. Validation loss: 0.1120.. Training loss: 0.1082.. 0.0297 s/batch
Epoch: 9/100.. Validation loss: 0.1124.. Training loss: 0.1066.. 0.0292 s/batch
Epoch: 10/100.. Validation loss: 0.1073.. Training loss: 0.0982.. 0.0303 s/batch
Epoch: 11/100.. Validation loss: 0.1082.. Training loss: 0.0946.. 0.0337 s/batch
Epoch: 12/100.. Validation loss: 0.1071.. Training loss: 0.0936.. 0.0322 s/batch
Epoch: 13/100.. Validation loss: 0.10

In [ ]:
epoch = np.arange(1,epochs+1)
plt.figure(figsize=(12, 7))
plt.rc('font', family='serif')
plt.rc('font', size=20)

plt.plot(epoch,loss_train,label='Training loss')
plt.plot(epoch,loss_val,c='k',label='Validation loss')
plt.xlabel('epoch')
plt.ylabel('MSE error')
plt.legend()
plt.savefig(r'./figures/loss_speed.pdf', bbox_inches='tight')

In [ ]:
torch.save(lstm.state_dict(), 'speed_ND_2.pt')